In [3]:
import sys
import os
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
%run dataset.ipynb 
%run neuralnet.ipynb
%run training.ipynb
%run getgraph.ipynb

Using TensorFlow backend.


In [17]:
# Configuration parameters
parser = argparse.ArgumentParser(usage='A training program for classifying the EMNIST dataset')
parser.add_argument('-f', '--file', type=str, help='Path .mat file data', required=True)
parser.add_argument('--width', type=int, default=28, help='Width of the images')
parser.add_argument('--height', type=int, default=28, help='Height of the images')
parser.add_argument('--max', type=int, default=None, help='Max amount of data to use')
parser.add_argument('--epochs', type=int, default=18, help='Number of epochs to train on')
parser.add_argument('--verbose', action='store_true', default=False, help='Enables verbose printing')

#     args = parser.parse_args()

data_file = 'emnist-balanced.mat'
# data_file = 'emnist-letters.mat' # This data set has starting index at 1.... WTF

data_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/data/' + data_file
#     args = parser.parse_args(['-f', data_path, '--max', '500'])
args = parser.parse_args(['-f', data_path])

bin_dir = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin'
if not os.path.exists(bin_dir):
    os.makedirs(bin_dir)

In [18]:
# Construct training data
training_data = load_data(args.file, width=args.width, height=args.height, max_data=args.max, verbose=args.verbose)

In [ ]:
# Construct model
model = build_net(training_data, width=args.width, height=args.height, verbose=args.verbose)

In [ ]:
# Train
train(model, training_data, epochs=args.epochs)

In [19]:
(x_train, y_train), (x_test, y_test), mapping, nb_classes = training_data

In [ ]:

y_test = np_utils.to_categorical(y_test, nb_classes)

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [33]:
# Generating labels file
labels = '\n'.join([chr(v) for v in mapping.values()])
with open("labels.txt", "w") as label_file:
    label_file.write(labels)


[[ (array([[ (array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[23],
       [ 7],
       [16],
       ..., 
       [13],
       [15],
       [19]], dtype=uint8), array([[1234],
       [ 799],
       [3444],
       ..., 
       [ 785],
       [ 208],
       [ 875]], dtype=uint16))]],
      dtype=[('images', 'O'), ('labels', 'O'), ('writers', 'O')]), array([[ (array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[ 1],
       [ 1],
       [ 1],
       ..., 
       [26],
       [26],
       [26]], dtype=uint8), array([[3259],
       [3984],
       [2060],
       ..., 
       [1888],
       [3407],
       [1301]], dtype=uint16))]],
      dtype=[